In [23]:
import numpy as np
import pandas as pd
import sympy

import weldx
import weldx.measurement as msm
import weldx.transformations as tf
from weldx import Q_
from weldx import util

In [24]:
import ipywidgets as widgets
from IPython.display import display
import xarray as xr
import io

In [25]:
class CsvMappingFormGroup:
    
    def __init__(self, form_title="Unbekannte Messreihe", unit_value=""):
        
        title = widgets.HTML(value="<h3>" + form_title + "</h3>")
        file = widgets.FileUpload(multiple=False)
        out = widgets.Output()
        
        def on_file_change(e):
            out.clear_output()

            keys = list(e["new"].keys())
            content = list(e["new"].values())[0]["content"]
            with out:
                print(keys)
                df = pd.read_csv(io.BytesIO(content), nrows=5)
            with out:
                display(df)
                cols = list(df.dtypes)

            with out:
                cols = list(df.columns) + ["keine Zuweisung"]
                target_cols = ["time", "value"]
                col_mapping_dropdowns = []
                form_group = []

                for i in range(0, len(target_cols)):
                    drop_down = widgets.Dropdown(
                        options=cols,
                        value=cols[i if i < len(cols) else 0],
                        disabled=False,
                     )
                    col_mapping_dropdowns.append(drop_down)
                    form_group = form_group + [widgets.Label(target_cols[i] + "-Spalte"), drop_down]

                unit = widgets.Text(value=unit_value)
                form_group.append(widgets.Label("Einheit"))
                form_group.append(unit)
                convert_button = widgets.Button(icon="file-export", description="Konvertieren")
                form_group.append(convert_button)
                validation_out = widgets.Output()
                form_group.append(validation_out)
                def on_click(e):
                    validation_out.clear_output()
                    try:
                        time_column = col_mapping_dropdowns[0].value
                        value_column = col_mapping_dropdowns[1].value

                        values = Q_(np.array(df[value_column]), unit.value)
                        da = xr.DataArray(values, coords=[pd.to_datetime(df[time_column])], dims=["time"])
                        self.value = da
                        with validation_out:
                            display(self.value)
                    except Exception as e:
                        with validation_out:
                            print(e)


                convert_button.on_click(on_click)

                form_group_box = widgets.VBox(form_group)
                display(form_group_box)
        file.observe(on_file_change, names='value')
        
        self.title = title
        self.file = file
        self.out = out
        self.value = []
    def build_widget_box(self):
        return widgets.VBox([self.title, self.file, self.out])
    
    def weldx_measurement_data(self, name="Unknown"):      
        data = msm.Data(name=name, data=self.value)
        return data

In [36]:
form_title = widgets.HTML(value="<h2>Messreihen erfassen</h2>")
fg_I = CsvMappingFormGroup("1.) Stromstärke", "ampere")
fg_U = CsvMappingFormGroup("2.) Spannung", "volt")

In [37]:
form = widgets.VBox([form_title ,fg_I.build_widget_box(), fg_U.build_widget_box()])
form

In [46]:
import asdf

save_out = widgets.Output()
def on_save(e):
    measurement_data = [fg_I.weldx_measurement_data("Welding current"), fg_U.weldx_measurement_data("Welding voltage")]
    tree = {
        "data": measurement_data
    }
    af = asdf.AsdfFile(tree) 
    af.write_to('measuremen_data.asdf')
    buffer = weldx.asdf.util._write_buffer(tree)
    
    save_out.clear_output()
    with save_out:
        display(weldx.asdf.util.notebook_fileprinter(buffer))

save_button = widgets.Button(icon="save", description="Speichern")
save_button.on_click(on_save)
display(save_button, save_out)

Button(description='Speichern', icon='save', style=ButtonStyle())

Output()